# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine  
from nltk.tokenize import word_tokenize
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pickle

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henrikwarfvinge/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster-data.db')


In [4]:
df = pd.read_sql('SELECT * FROM messages', engine)

In [5]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    
    #remove punctuation and make lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize the text
    words = word_tokenize(text)
    
    #remove stopwords
    clean_words = []
    for w in words:
        if w not in stopwords.words("english"):
            clean_words.append(w)
    
    #Stem word tokens and remove stop words
    #stemmer = PorterStemmer()
    #stop_words = stopwords.words("english")
    
    #words_final = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    
    words_final = []
    for w in clean_words:
        clean_tok = lemmatizer.lemmatize(w).strip()
        words_final.append(clean_tok)
    
    return words_final

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# train classifier
np.random.seed(9)
pipeline.fit(X_train, y_train)


/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: T

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [10]:
# predict on test data
y_test_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
col_names = list(y.columns)

In [12]:
actual = y_test
actual[:5]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3694,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13251,1,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
16506,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [13]:
predicted = y_test_pred
predicted[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [15]:
print(classification_report(y_test, y_test_pred, target_names=col_names))

                        precision    recall  f1-score   support

               related       0.85      0.91      0.88      4967
               request       0.80      0.45      0.58      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.74      0.61      0.67      2679
          medical_help       0.57      0.10      0.17       519
      medical_products       0.64      0.10      0.17       321
     search_and_rescue       0.17      0.01      0.01       153
              security       0.50      0.01      0.02       124
              military       0.63      0.08      0.14       223
                 water       0.80      0.38      0.52       416
                  food       0.81      0.50      0.62       724
               shelter       0.81      0.31      0.45       579
              clothing       0.75      0.06      0.11        97
                 money       0.62      0.04      0.07       141
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
parameters = {
              'clf__estimator__n_estimators':[25, 100]
              }

#'vect__min_df': [1, 4],
#'tfidf__use_idf':[True, False],
#'clf__estimator__max_depth' : [10, 20, 50]

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_weighted', verbose = 3)

grid_model = cv.fit(X_train, y_train)


/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=25 .................................


/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV] ..... clf__estimator__n_estimators=25, score=0.569, total= 2.1min
[CV] clf__estimator__n_estimators=25 .................................


/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.3min remaining:    0.0s


...

[CV] .... clf__estimator__n_estimators=100, score=0.572, total= 4.6min
[CV] clf__estimator__n_estimators=100 ................................


/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] .... clf__estimator__n_estimators=100, score=0.573, total= 4.9min
[CV] clf__estimator__n_estimators=100 ................................


/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 20.8min finished


[CV] .... clf__estimator__n_estimators=100, score=0.564, total= 5.0min


In [17]:
grid_model.cv_results_

{'mean_fit_time': array([ 95.394617  , 235.40941683]),
 'std_fit_time': array([1.67859348, 9.00348813]),
 'mean_score_time': array([31.65268373, 53.50646432]),
 'std_score_time': array([1.3881254, 2.4654758]),
 'param_clf__estimator__n_estimators': masked_array(data=[25, 100],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__estimator__n_estimators': 25},
  {'clf__estimator__n_estimators': 100}],
 'split0_test_score': array([0.56922956, 0.57196156]),
 'split1_test_score': array([0.57320518, 0.57305944]),
 'split2_test_score': array([0.56194312, 0.56382653]),
 'mean_test_score': array([0.56812596, 0.56961585]),
 'std_test_score': array([0.00466347, 0.00411813]),
 'rank_test_score': array([2, 1], dtype=int32)}

In [18]:
grid_model.best_params_

{'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
y_pred = cv.predict(X_test)

In [20]:
print(classification_report(y_test, y_pred, target_names=col_names))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4967
               request       0.82      0.50      0.63      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.75      0.69      0.72      2679
          medical_help       0.59      0.08      0.14       519
      medical_products       0.80      0.14      0.24       321
     search_and_rescue       0.33      0.01      0.01       153
              security       0.50      0.01      0.02       124
              military       0.75      0.07      0.12       223
                 water       0.85      0.42      0.56       416
                  food       0.82      0.64      0.72       724
               shelter       0.80      0.40      0.54       579
              clothing       0.81      0.13      0.23        97
                 money       0.75      0.06      0.12       141
        missing_people       0.00      

/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [21]:
grid_model_pred_test = grid_model.predict(X_test)

In [22]:
print(classification_report(y_test, grid_model_pred_test , target_names=col_names))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4967
               request       0.82      0.50      0.63      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.75      0.69      0.72      2679
          medical_help       0.59      0.08      0.14       519
      medical_products       0.80      0.14      0.24       321
     search_and_rescue       0.33      0.01      0.01       153
              security       0.50      0.01      0.02       124
              military       0.75      0.07      0.12       223
                 water       0.85      0.42      0.56       416
                  food       0.82      0.64      0.72       724
               shelter       0.80      0.40      0.54       579
              clothing       0.81      0.13      0.23        97
                 money       0.75      0.06      0.12       141
        missing_people       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [23]:
# Try using a SVM model instead of Random Forest Classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

parameters2 = {'vect__min_df': [5],
              'tfidf__use_idf':[True],
              'clf__estimator__kernel': ['poly'], 
              'clf__estimator__degree': [2, 3],
              'clf__estimator__C':[10, 100]}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2, scoring = 'f1_weighted', verbose = 3)

# Find best parameters
np.random.seed(9)
grid_model2 = cv2.fit(X_train, y_train)

/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__C=10, clf__estimator__degree=2, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.0min remaining:    0.0s


[CV]  clf__estimator__C=10, clf__estimator__degree=2, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.207, total= 4.0min
[CV] clf__estimator__C=10, clf__estimator__degree=2, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.4min remaining:    0.0s


...

/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__C=100, clf__estimator__degree=3, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.211, total= 3.3min
[CV] clf__estimator__C=100, clf__estimator__degree=3, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5 


/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 49.7min finished


[CV]  clf__estimator__C=100, clf__estimator__degree=3, clf__estimator__kernel=poly, tfidf__use_idf=True, vect__min_df=5, score=0.207, total= 3.7min


/miniconda3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [24]:
grid_model2_pred_test = grid_model2.predict(X_test)

In [25]:
print(classification_report(y_test, grid_model2_pred_test , target_names=col_names))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      4967
               request       0.00      0.00      0.00      1120
                 offer       0.00      0.00      0.00        23
           aid_related       0.00      0.00      0.00      2679
          medical_help       0.00      0.00      0.00       519
      medical_products       0.00      0.00      0.00       321
     search_and_rescue       0.00      0.00      0.00       153
              security       0.00      0.00      0.00       124
              military       0.00      0.00      0.00       223
                 water       0.00      0.00      0.00       416
                  food       0.00      0.00      0.00       724
               shelter       0.00      0.00      0.00       579
              clothing       0.00      0.00      0.00        97
                 money       0.00      0.00      0.00       141
        missing_people       0.00      

/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [29]:
# Pickle best model
pickle.dump(grid_model, open('disaster_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.